In [1]:
import sys
import os
from time import clock, ctime
import numpy as np
from scipy.sparse import coo_matrix
import sqlite3
from os.path import join
import pandas as pd

In [2]:
sys.path.append('/media/pedro/storage/src/aequilibrae')
from aequilibrae.paths import Graph, NetworkSkimming, SkimResults, PathResults, path_computation
from aequilibrae.matrix import AequilibraeMatrix

In [3]:
fldr = '/media/pedro/server/Pedro/Professional/VLC/Projects/Vancouver'
aeg_pth = '/media/pedro/server/Pedro/Professional/VLC/Projects/Vancouver/Network/graph.aeg'
trunk_distances = '/media/pedro/server/Pedro/Professional/VLC/Projects/Vancouver/Network/trunk_distances.aem'

In [4]:
g = Graph()
g.load_from_disk(aeg_pth)
g.prepare_graph(g.all_nodes)
g.set_graph(cost_field='length', skim_fields=['timau'],block_centroid_flows=False)

result = SkimResults()
result.prepare(g)

skm = NetworkSkimming(g, result)
skm.execute()

In [5]:
trunk = AequilibraeMatrix()
trunk.load(trunk_distances)
aux = coo_matrix(trunk.length)
z = int(trunk.index.max() + 1)

trunk = coo_matrix((aux.data, (trunk.index[aux.row], trunk.index[aux.col])), shape=(z, z))
trunk = trunk.toarray().astype(np.float32)

In [6]:
# We create the sparse_matrix that we will use to generate the routing graph
idx = result.skims.index[:]
mat = coo_matrix(result.skims.matrix['length'])
mat = coo_matrix((mat.data, (idx[mat.row], idx[mat.col])), shape=(int(idx.max()) + 1, int(idx.max()) + 1))
length = mat.toarray().astype(np.float32)
del(mat)

In [7]:
nodelyr = sqlite3.connect(os.path.join(fldr, "Network/final_layer.gpkg"))
nds = pd.read_sql_query("SELECT fid FROM nodes where park_and_ride=1", nodelyr).fid.tolist()


In [8]:
tripfile = sqlite3.connect(os.path.join(fldr, "trip_table.sqlite"))
cur = tripfile.cursor()
cur.execute('select distinct "class" from distributed_trips')
classes = cur.fetchall()

In [26]:
closest = np.zeros_like(length)
closest.fill(np.inf)

for i in nds:
    closest[i, :] = length[i, :]
    closest[:, i] = length[:, i]

In [71]:
def trip_chaining(trip):
    min_dist_to_chain = 5
    max_perc_increase = 0.75
    min_perc_whole_trip = 0.4
    
    x = trip.from_node
    y = trip.to_node
    t = trip.departure_minute
    
    dist = length[x, y]
    
    original_trip = [[trip.name, trip['class'], trip.flow, x, y, t]]
    
    if dist < min_dist_to_chain:
        return original_trip
    
    a = np.argmin(closest[x, :], axis=0)
    b = np.argmin(closest[y, :], axis=0)

    access = length[x, a]
    eggress = length[b, y]
    middle = trunk[a, b]
    new_tot = access + eggress + middle
    
    if new_tot / dist > 1 + max_perc_increase:
        return original_trip
    
    if middle / new_tot < min_perc_whole_trip:
        return original_trip
    
    
    # Average of 25kh/m in minutes
    new_t = t + ((access + middle) / 25) * 60 
    return [[trip.name, trip['class'], trip.flow, x, a, t], 
            [trip.name, trip['class'], trip.flow, b, y, new_t]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less


CPU times: user 4.45 s, sys: 0 ns, total: 4.45 s
Wall time: 4.44 s


In [73]:
len(all_trips), trips.shape[0]

(40109, 33993)

In [89]:
%%time

outdb = sqlite3.connect(os.path.join(fldr, 'connected_trips.sqlite'))
outdb.execute('drop table if exists trips')
for c in classes:
    print(c, "  --  ", ctime())
    trips = pd.read_sql_query('SELECT * FROM distributed_trips where "class"="{}"'.format(c[0]), tripfile)
    trips.drop(['index'], axis=1, inplace=True)
    all_trips = []
    for trip in trips.iterrows():
        all_trips.extend(trip_chaining(trip[1]))

    df = pd.DataFrame(all_trips, columns = ['orig_trip_fid', 'class', 'flow', 'from_node', 
                                            'to_node', 'departure_minute'])
    df.to_sql('trips', outdb, if_exists='append')

('Bk_pertrp_Am',)   --   Wed Jun 19 12:21:03 2019


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less


('Bk_pertrp_Md',)   --   Wed Jun 19 12:21:13 2019
('Bk_pertrp_Pm',)   --   Wed Jun 19 12:21:22 2019


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in float_scalars


('HOV_drvtrp_VOT_1_Am',)   --   Wed Jun 19 12:21:34 2019
('HOV_drvtrp_VOT_1_Md',)   --   Wed Jun 19 12:21:55 2019
('HOV_drvtrp_VOT_1_Pm',)   --   Wed Jun 19 12:22:10 2019
('HOV_drvtrp_VOT_2_Am',)   --   Wed Jun 19 12:22:26 2019
('HOV_drvtrp_VOT_2_Md',)   --   Wed Jun 19 12:22:46 2019
('HOV_drvtrp_VOT_2_Pm',)   --   Wed Jun 19 12:23:01 2019
('HOV_drvtrp_VOT_3_Am',)   --   Wed Jun 19 12:23:27 2019
('HOV_drvtrp_VOT_3_Md',)   --   Wed Jun 19 12:23:41 2019
('HOV_drvtrp_VOT_3_Pm',)   --   Wed Jun 19 12:23:57 2019
('HOV_pertrp_VOT_1_Am',)   --   Wed Jun 19 12:24:20 2019
('HOV_pertrp_VOT_1_Md',)   --   Wed Jun 19 12:25:07 2019
('HOV_pertrp_VOT_1_Pm',)   --   Wed Jun 19 12:25:38 2019
('HOV_pertrp_VOT_2_Am',)   --   Wed Jun 19 12:26:13 2019
('HOV_pertrp_VOT_2_Md',)   --   Wed Jun 19 12:26:55 2019
('HOV_pertrp_VOT_2_Pm',)   --   Wed Jun 19 12:27:28 2019
('HOV_pertrp_VOT_3_Am',)   --   Wed Jun 19 12:28:22 2019
('HOV_pertrp_VOT_3_Md',)   --   Wed Jun 19 12:28:46 2019
('HOV_pertrp_VOT_3_Pm',)   --  